In [ ]:
import sys
import traceback
import asyncio
import nest_asyncio
from threading import Thread
from arduino_iot_cloud import ArduinoCloudClient
import csv
import os.path
from datetime import datetime

# Enable nested asyncio loops to allow for the proper handling of asynchronous tasks
nest_asyncio.apply()

DEVICE_ID = "e5ea5829-0f0a-4e43-b462-c6c9c9df5d8f"
SECRET_KEY = "hS6Ty2@3?NcNGF5xVelU?L0nq"

# CSV file names
COMBINED_CSV = 'Python_Accelerometer_Combined.csv'

# Global variable to store the latest values
latest_values = {
    'X': None,
    'Y': None,
    'Z': None,
    'timestamp': None
}

# Returns timestamp of now in a human-readable format
def get_time_stamp() -> str:
    now = datetime.now()  # now stores current time
    timeStamp = now.strftime('%Y-%m-%d %H:%M:%S')  # Format timestamp as YYYY-MM-DD HH:MM:SS
    print(f"Generated timestamp: {timeStamp}")  # Debug print
    return timeStamp

# Checks if file exists and creates it
def check_file_exist(filename: str) -> None:
    # Check if file path exists
    if not os.path.isfile(filename):
        # Create field header names
        fields = ['timestamp', 'X_value', 'Y_value', 'Z_value']
        # Create a CSV file
        with open(filename, 'w', newline='') as csvFile:
            writer = csv.DictWriter(csvFile, fieldnames=fields)
            writer.writeheader()

# Prepares values to be saved and saves to a CSV file
def save_changed_values(filename: str) -> None:
    if latest_values['X'] is not None and latest_values['Y'] is not None and latest_values['Z'] is not None:
        # Prepare data for writing to CSV
        newData = [[latest_values['timestamp'], latest_values['X'], latest_values['Y'], latest_values['Z']]]
        # Add data to CSV file
        with open(filename, 'a', newline='') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerows(newData)
        print(f"Saved data: {newData}")  # Debug print

# Callback functions for each axis
def on_x_changed(client, value: float) -> None:
    print(f'New X value: {value}')
    latest_values['X'] = value
    latest_values['timestamp'] = get_time_stamp()
    save_changed_values(COMBINED_CSV)

def on_y_changed(client, value: float) -> None:
    print(f'New Y value: {value}')
    latest_values['Y'] = value
    latest_values['timestamp'] = get_time_stamp()
    save_changed_values(COMBINED_CSV)

def on_z_changed(client, value: float) -> None:
    print(f'New Z value: {value}')
    latest_values['Z'] = value
    latest_values['timestamp'] = get_time_stamp()
    save_changed_values(COMBINED_CSV)

# Asynchronous function to run the Arduino Cloud client
async def run_client():
    print("Connect to Client")

    client = ArduinoCloudClient(device_id=DEVICE_ID, username=DEVICE_ID, password=SECRET_KEY)
    client.register("acc_X", value=None, on_write=on_x_changed)
    client.register("acc_Y", value=None, on_write=on_y_changed)
    client.register("acc_Z", value=None, on_write=on_z_changed)
    
    # Providing interval and backoff arguments
    await client.run(interval=5, backoff=2)

# Run the asynchronous function in a new thread
def start_client():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(run_client())

if __name__ == "__main__":
    try:
        check_file_exist(COMBINED_CSV)
        
        # Start the client in a new thread
        thread = Thread(target=start_client)
        thread.start()
        
        # Main thread can continue with other tasks if needed
        thread.join()  # Wait for the thread to finish if you want synchronous execution
        
    except Exception as e:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback.print_tb(exc_traceback, file=sys.stdout)


Connect to Client


ERROR:root:task: connection_task raised exception: .
ERROR:root:task: discovery raised exception: .


New X value: -0.0069580078125
Generated timestamp: 2024-09-03 11:32:31
New Y value: -0.0320587158203125
Generated timestamp: 2024-09-03 11:32:31
New Z value: -0.99560546875
Generated timestamp: 2024-09-03 11:32:31
Saved data: [['2024-09-03 11:32:31', -0.0069580078125, -0.0320587158203125, -0.99560546875]]
New X value: -0.0077362060546875
Generated timestamp: 2024-09-03 11:32:32
Saved data: [['2024-09-03 11:32:32', -0.0077362060546875, -0.0320587158203125, -0.99560546875]]
New Y value: -0.0318603515625
Generated timestamp: 2024-09-03 11:32:33
Saved data: [['2024-09-03 11:32:33', -0.0077362060546875, -0.0318603515625, -0.99560546875]]
New Z value: -0.9957733154296875
Generated timestamp: 2024-09-03 11:32:34
Saved data: [['2024-09-03 11:32:34', -0.0077362060546875, -0.0318603515625, -0.9957733154296875]]
New X value: -0.0078887939453125
Generated timestamp: 2024-09-03 11:32:35
Saved data: [['2024-09-03 11:32:35', -0.0078887939453125, -0.0318603515625, -0.9957733154296875]]
New Y value: -0

ERROR:root:task: mqtt_task raised exception: [Errno 32] Broken pipe.


New Y value: -0.032684326171875
Generated timestamp: 2024-09-03 11:32:48
Saved data: [['2024-09-03 11:32:48', -0.006378173828125, -0.032684326171875, -0.9963226318359375]]


...

ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been closed (EOF) (_ssl.c:2483).
ERROR:root:task: connection_task raised exception: .


New X value: -0.007843017578125
Generated timestamp: 2024-09-03 11:35:57
Saved data: [['2024-09-03 11:35:57', -0.007843017578125, -0.0300445556640625, -0.995941162109375]]
New Y value: -0.0327911376953125
Generated timestamp: 2024-09-03 11:35:58
Saved data: [['2024-09-03 11:35:58', -0.007843017578125, -0.0327911376953125, -0.995941162109375]]
New Z value: -0.995147705078125
Generated timestamp: 2024-09-03 11:35:59
Saved data: [['2024-09-03 11:35:59', -0.007843017578125, -0.0327911376953125, -0.995147705078125]]


ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been closed (EOF) (_ssl.c:2483).
ERROR:root:task: connection_task raised exception: .


New X value: 0.2221221923828125
Generated timestamp: 2024-09-03 11:36:10
Saved data: [['2024-09-03 11:36:10', 0.2221221923828125, -0.0327911376953125, -0.995147705078125]]
New Y value: 0.004547119140625
Generated timestamp: 2024-09-03 11:36:11
Saved data: [['2024-09-03 11:36:11', 0.2221221923828125, 0.004547119140625, -0.995147705078125]]
New Z value: -0.976959228515625
Generated timestamp: 2024-09-03 11:36:12
Saved data: [['2024-09-03 11:36:12', 0.2221221923828125, 0.004547119140625, -0.976959228515625]]


ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been closed (EOF) (_ssl.c:2483).
ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been closed (EOF) (_ssl.c:2483).
ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been closed (EOF) (_ssl.c:2483).
ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been closed (EOF) (_ssl.c:2483).
ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been closed (EOF) (_ssl.c:2483).
ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been closed (EOF) (_ssl.c:2483).
ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: TLS/SSL connection has been clo